In [1]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 34.9 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, Audio
#https://huggingface.co/datasets/erenfazlioglu/turkishvoicedataset
#dataset = load_dataset("erenfazlioglu/turkishvoicedataset", split="train")
dataset = load_dataset("abbbinav/telugu_pal", split="train")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/357 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/243M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1956 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1956
})

In [4]:
# Calculate the number of rows for half the dataset
half_size = len(dataset) // 16

# Select the first half of the dataset
dataset = dataset.select(range(half_size))

print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 122
})


In [5]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [6]:
from transformers import SpeechT5Processor
checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
tokenizer = processor.tokenizer

In [8]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

In [9]:
dataset_vocab - tokenizer_vocab

{' ',
 'ం',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఎ',
 'ఏ',
 'ఒ',
 'క',
 'ఖ',
 'గ',
 'చ',
 'జ',
 'ట',
 'ఠ',
 'డ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ',
 'ా',
 'ి',
 'ీ',
 'ు',
 'ూ',
 'ృ',
 'ె',
 'ే',
 'ై',
 'ొ',
 'ో',
 'ౌ',
 '్'}

In [10]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (except apostrophes)
    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['sentence'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

{'audio': [{'path': None, 'array': array([-1.86177203e-07,  2.44308467e-07, -3.05532012e-07, ...,
       -1.24657818e-05, -2.84376001e-05, -3.37785386e-05]), 'sampling_rate': 16000}, {'path': None, 'array': array([-1.05459549e-07,  1.46093953e-07, -1.89618731e-07, ...,
       -3.00593856e-05, -3.02350818e-05, -3.19244937e-05]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 8.80536390e-08, -1.08087988e-07,  1.28658030e-07, ...,
       -1.14846625e-06,  3.13413329e-06, -9.77882883e-06]), 'sampling_rate': 16000}], 'sentence': ['ఆ అబ్బాయి చెడ్డవాడు.', 'ఆత్మరక్షణకు మనిషి అణ్వాయుధాలను కనిపెట్టాడు.', 'నైకమత్యమే మహాబలమ్ము'], 'normalized_text': ['ఆ అబబయ చడడవడ', 'ఆతమరకషణక మనష అణవయధలన కనపటటడ', 'నకమతయమ మహబలమమ']}


In [11]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

In [12]:
dataset_vocab - tokenizer_vocab

{' ',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఎ',
 'ఏ',
 'ఒ',
 'క',
 'ఖ',
 'గ',
 'చ',
 'జ',
 'ట',
 'ఠ',
 'డ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ'}

In [13]:
replacements = [
    ("అ", "a"),   # Short a
    ("ఆ", "aa"),  # Long a
    ("ఇ", "i"),   # Short i
    ("ఈ", "ee"),  # Long i
    ("ఉ", "u"),   # Short u
    ("ఊ", "oo"),  # Long u
    ("ఎ", "e"),   # Short e
    ("ఏ", "ee"),  # Long e
    ("ఐ", "ai"),  # ai diphthong
    ("ఒ", "o"),   # Short o
    ("క", "ka"),  # Ka
    ("ఖ", "kha"), # Kha
    ("గ", "ga"),  # Ga
    ("ఘ", "gha"), # Gha
    ("చ", "cha"), # Cha
    ("జ", "ja"),  # Ja
    ("ఞ", "nya"), # Nya (nasal)
    ("ట", "ta"),  # Retroflex Ta
    ("ఠ", "tha"), # Aspirated Ta
    ("డ", "da"),  # Retroflex Da
    ("ణ", "na"),  # Retroflex Na
    ("త", "ta"),  # Dental Ta
    ("థ", "tha"), # Aspirated Dental Ta
    ("ద", "da"),  # Dental Da
    ("ధ", "dha"), # Aspirated Da
    ("న", "na"),  # Na
    ("ప", "pa"),  # Pa
    ("ఫ", "pha"), # Pha
    ("బ", "ba"),  # Ba
    ("భ", "bha"), # Bha
    ("మ", "ma"),  # Ma
    ("య", "ya"),  # Ya
    ("ర", "ra"),  # Ra
    ("ల", "la"),  # La
    ("ళ", "lla"), # Retroflex Lla
    ("వ", "va"),  # Va
    ("శ", "sha"), # Sha
    ("ష", "ssha"),# Retroflex Sha
    ("స", "sa"),  # Sa
    ("హ", "ha"),  # Ha
]


def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

In [14]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [15]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [16]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())


['input_ids', 'labels', 'speaker_embeddings']

In [17]:
processed_example["speaker_embeddings"].shape

(512,)

In [18]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)


Map:   0%|          | 0/122 [00:00<?, ? examples/s]

In [19]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

Filter:   0%|          | 0/122 [00:00<?, ? examples/s]

122

In [20]:
dataset = dataset.train_test_split(test_size=0.1)

In [21]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [22]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [23]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [24]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [25]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_telugu",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [29]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.515500,0.513108
200,0.447000,0.471814
300,0.415000,0.462820
400,0.400900,0.466117
500,0.393200,0.483162


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/e

TrainOutput(global_step=500, training_loss=0.46183933639526364, metrics={'train_runtime': 1042.2996, 'train_samples_per_second': 15.351, 'train_steps_per_second': 0.48, 'total_flos': 817218560746368.0, 'train_loss': 0.46183933639526364, 'epoch': 142.85714285714286})

In [30]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


events.out.tfevents.1729777247.31be1d4a2314.1273.0:   0%|          | 0.00/18.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/minnuhh/speecht5_finetuned_telugu/commit/0b2c42482e369fdc38da7e5b1b47db33354223e7', commit_message='End of training', commit_description='', oid='0b2c42482e369fdc38da7e5b1b47db33354223e7', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "minnuhh/speecht5_finetuned_telugu"
)

config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [34]:
example = dataset["test"][10]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [44]:
text = "aa mandahasamani bhavamu"


In [45]:
number_words = {
    0: "సున్నా", 1: "ఒకటి", 2: "రెండు", 3: "మూడు", 4: "నాలుగు", 5: "ఐదు", 6: "ఆరు", 7: "ఏడు", 8: "ఎనిమిది", 9: "తొమ్మిది",
    10: "పది", 11: "పదకొండు", 12: "పన్నెండు", 13: "పదమూడు", 14: "పద్నాలుగు", 15: "పదిహేను", 16: "పదహారు", 17: "పదిహేడు",
    18: "పద్దెనిమిది", 19: "పంతొమ్మిది", 20: "ఇరవై"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + (" " + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " వంద" if hundreds > 1 else "వంద") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return (number_to_words(thousands) + " వెయ్యి" if thousands > 1 else "వెయ్యి") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " మిలియన్" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " బిలియన్" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

import re

def replace_numbers_with_words(text):

    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and change them to words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result


In [46]:
# Function to clean up text using the replacement pairs
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [47]:
converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = normalize_text(cleaned_text)
final_text

'aa mandahasamani bhavamu'

In [48]:
inputs = processor(text=final_text, return_tensors="pt")

In [49]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)


In [50]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output.wav', speech.numpy(), 16000)


In [51]:
from transformers import SpeechT5Config, SpeechT5ForTextToSpeech

config = SpeechT5Config.from_pretrained("microsoft/speecht5_tts", dropout=0.1)  # 10% dropout
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts", config=config)


In [54]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_telugu",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [55]:
from transformers import EarlyStoppingCallback

# Add Early Stopping Callback to your Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop after 3 bad evaluations
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [63]:
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan
import torch
import numpy as np
from IPython.display import Audio

# Load your fine-tuned model and processor
model = SpeechT5ForTextToSpeech.from_pretrained("speecht5_finetuned_telugu")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Test with technical terms and phrases
test_phrases = ["API", "OAuth", "REST", "CUDA"]

# Example speaker embeddings: Can use any embeddings if your dataset doesn't provide them.
# If you have specific embeddings, load them. Otherwise, create a random embedding for testing.
speaker_embeddings = torch.randn(1, 512)  # Dummy speaker embedding for testing (512-dimensional)

for phrase in test_phrases:
    print(f"Generating speech for: {phrase}")

    # Tokenize the text
    inputs = processor(text=phrase, return_tensors="pt")

    # Generate speech output using the fine-tuned model and the vocoder
    with torch.no_grad():
        speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    # Convert speech to numpy for playback
    speech_numpy = speech.cpu().numpy()

    # Play the audio
    display(Audio(speech_numpy, rate=16000))  # Ensure correct sample rate (16 kHz for TTS)


Generating speech for: API


Generating speech for: OAuth


Generating speech for: REST


Generating speech for: CUDA
